In [1]:
import os
import numpy as np
import sympy as sp
import torch

In [2]:
cd ../

/home/kpgelvan/SymbolicMathematics


In [3]:
from src.utils import AttrDict
from src.envs import build_env
from src.model import build_modules

from src.utils import to_cuda
from src.envs.sympy_utils import simplify

In [4]:
ls

CODE_OF_CONDUCT.md  main.py                          sbatch_scripts/
CONTRIBUTING.md     notebooks/                       sbatch_scripts_eval/
data/               preprocess_build_relmat_dict.py  sbatch_scripts_generate/
dumped/             preprocess_build_relmat.py       script_preprocess.sbatch
LICENSE             README.md                        split_data.py
logs/               requirements/                    src/


In [5]:
!wc -l 'data/ode1.train'

65340431 data/ode1.train


In [7]:
65_340_431

65340431

In [17]:
with open('data/ode1.train', 'r') as train_file:
    for i, line in enumerate(train_file):
        if line.split('|')[1][:3] != 'add':
            print(i, line)

104 19931|Y'	mul pow x INT- 1 add x mul a8 x

106 19669|Y'	mul x pow add x mul a8 x INT- 1

923 3601|Y'	mul x pow add mul INT+ 2 x mul a8 x INT- 1

928 3574|Y'	mul pow x INT- 1 add mul INT+ 2 x mul a8 x

1192 2876|Y'	mul pow x INT- 1 add mul INT- 1 x mul a8 x

1207 2830|Y'	mul x pow add mul INT- 1 x mul a8 x INT- 1

1733 1978|mul pow add INT+ 1 cos mul INT+ 2 x INT- 1 add mul INT- 1 sin mul INT+ 2 x add mul INT- 2 x add mul INT+ 2 Y mul Y' sin mul INT+ 2 x	add x mul a8 pow tan x INT- 1

2025 1686|Y'	mul INT+ 2 mul x pow add x mul a8 x INT- 1

2404 1349|Y'	mul div INT+ 1 INT+ 2 mul pow x INT- 1 add x mul a8 x

3415 1053|mul INT+ 3 mul pow abs x div INT- 2 INT+ 3 add mul INT+ 6 mul pow x INT+ 2 pow mul pow x INT- 1 add mul x abs Y mul INT- 1 mul abs x Y div INT+ 2 INT+ 3 mul pow INT+ 2 div INT+ 2 INT+ 3 mul pow abs x div INT+ 2 INT+ 3 add mul INT- 1 mul x Y' add mul INT- 1 mul Y sign mul pow x INT- 1 Y add mul x mul Y' sign mul pow x INT- 1 Y Y	mul x pow add a8 x INT+ 3

4114 921|Y'	mul 

178773 21|mul add mul INT+ 2 Y add mul add mul INT- 2 Y add mul x Y' mul INT- 1 mul x Y exp add INT+ 1 0 mul INT+ 2 x add mul INT- 1 mul x Y' mul INT- 1 mul x Y exp add INT- 5 mul INT- 1 x	mul a8 mul pow x INT+ 2 sinh add INT+ 5 x

178804 21|Y'	mul x pow add mul INT+ 1 5 x mul a8 x INT- 1

178868 20|Y'	mul pow x INT- 1 acos cos mul a8 x

178894 20|mul pow cosh mul pow x INT- 1 mul exp mul INT- 1 x Y INT- 2 add mul x Y add mul INT- 1 mul x Y' add mul x mul pow cosh mul pow x INT- 1 mul exp mul INT- 1 x Y INT+ 2 mul exp x tanh mul pow x INT- 1 mul exp mul INT- 1 x Y Y	mul x mul atanh mul a8 x exp x

178901 20|mul add mul x Y' add mul x Y add mul add mul x Y' add mul INT- 1 mul x Y Y exp add INT+ 1 0 mul INT+ 2 x Y exp add INT- 5 mul INT- 1 x	mul a8 mul pow x INT- 1 cosh add INT+ 5 x

179015 20|Y'	mul pow x INT- 1 mul add x mul a8 x ln INT+ 4

179631 20|mul add mul add mul INT- 1 Y Y' exp mul INT+ 2 x mul add Y' Y exp INT+ 6 exp add INT- 3 mul INT- 1 x	mul a8 cosh add INT- 3 x

179666 20|

317937 12|Y'	sin exp mul x pow add x mul a8 x INT- 1

317938 12|mul add mul INT- 1 exp mul div INT+ 1 INT+ 2 Y add mul INT+ 4 exp mul div INT+ 1 INT+ 2 x mul Y' exp mul div INT+ 1 INT+ 2 Y exp mul div INT- 1 INT+ 2 x	add x ln pow add a8 mul INT+ 2 x INT+ 2

318071 12|mul INT+ 3 mul pow abs x div INT- 2 INT+ 3 add mul INT+ 1 2 mul pow x INT+ 2 pow mul pow x INT- 1 add mul x abs add pow x INT+ 2 mul INT- 1 Y add mul pow x INT+ 2 abs x mul INT- 1 mul abs x Y div INT+ 2 INT+ 3 mul pow INT+ 2 div INT+ 2 INT+ 3 mul pow abs x div INT+ 2 INT+ 3 add pow x INT+ 2 add mul pow x INT+ 2 sign mul pow x INT- 1 add pow x INT+ 2 mul INT- 1 Y add mul Y sign mul pow x INT- 1 add pow x INT+ 2 mul INT- 1 Y add mul INT- 1 mul x Y' add mul INT- 1 mul x mul Y' sign mul pow x INT- 1 add pow x INT+ 2 mul INT- 1 Y Y	mul x add x pow add a8 mul INT+ 2 x INT+ 3

318130 12|mul add mul pow Y INT+ 2 exp mul div INT+ 1 INT+ 2 exp x add mul pow Y INT+ 2 exp add mul div INT+ 1 INT+ 2 exp x mul INT+ 2 x mul INT+ 2 mul add

KeyboardInterrupt: 

In [18]:
!head 'data/ode1.train'

641556|add mul INT- 2 Y mul x Y'	mul a8 pow x INT+ 2
406394|add INT- 2 Y'	add a8 mul INT+ 2 x
265543|add mul INT- 3 Y mul x Y'	mul a8 pow x INT+ 3
238156|add INT+ 1 Y'	add a8 mul INT- 1 x
201768|add pow x INT+ 2 add mul INT- 1 mul x Y' Y	mul x add a8 x
161143|add x add mul INT- 2 Y mul x Y'	add x mul a8 pow x INT+ 2
160067|add INT- 3 Y'	add a8 mul INT+ 3 x
132622|add INT- 1 add mul INT- 2 x Y'	add a8 add x pow x INT+ 2
122995|add mul INT- 4 Y mul x Y'	mul a8 pow x INT+ 4
115829|add mul INT+ 2 Y mul x Y'	mul a8 pow x INT- 2


### Отрезаем половину

In [25]:
from tqdm import tqdm
with open('data/ode1.train', 'r') as f, open('data/ode1.train_half', 'w') as f_new:
    for i, line in tqdm(enumerate(f)):
        if i > (65_340_431 / 2):
            break
        else:
            f_new.write(line)


0it [00:00, ?it/s]
132168it [00:00, 1321637.15it/s]
254037it [00:00, 1288971.33it/s]
339525it [00:00, 1093049.67it/s]
425968it [00:00, 1007282.98it/s]
499969it [00:00, 890368.48it/s] 
622589it [00:00, 970063.92it/s]
746709it [00:00, 1038093.42it/s]
871748it [00:00, 1093796.79it/s]
994250it [00:00, 1112005.98it/s]
1113912it [00:01, 1136104.49it/s]
1226329it [00:01, 1095276.61it/s]
1335338it [00:01, 1061844.42it/s]
1441382it [00:01, 1058049.21it/s]
1547092it [00:01, 1014955.81it/s]
1648895it [00:01, 995177.54it/s] 
1748714it [00:01, 912282.39it/s]
1841502it [00:01, 802112.75it/s]
1941375it [00:01, 847193.08it/s]
2029235it [00:02, 826675.96it/s]
2142057it [00:02, 895968.08it/s]
2256767it [00:02, 958949.39it/s]
2370795it [00:02, 1006988.69it/s]
2485759it [00:02, 1045922.82it/s]
2601733it [00:02, 1077650.67it/s]
2717474it [00:02, 1100398.67it/s]
2833563it [00:02, 1117869.31it/s]
2950202it [00:02, 1131995.82it/s]
3067186it [00:02, 1143088.98it/s]
3184450it [00:03, 1151796.23it/s]
3300861it 

In [26]:
!wc -l data/ode1.train_half

32670216 data/ode1.train_half


In [27]:
!wc -l data/ode1.train

65340431 data/ode1.train



32663776it [00:54, 736643.32it/s]

### паста из кода авторов

In [7]:
OPERATORS = {
        # Elementary functions
        'add': 2,
        'sub': 2,
        'mul': 2,
        'div': 2,
        'pow': 2,
        'rac': 2,
        'inv': 1,
        'pow2': 1,
        'pow3': 1,
        'pow4': 1,
        'pow5': 1,
        'sqrt': 1,
        'exp': 1,
        'ln': 1,
        'abs': 1,
        'sign': 1,
        # Trigonometric Functions
        'sin': 1,
        'cos': 1,
        'tan': 1,
        'cot': 1,
        'sec': 1,
        'csc': 1,
        # Trigonometric Inverses
        'asin': 1,
        'acos': 1,
        'atan': 1,
        'acot': 1,
        'asec': 1,
        'acsc': 1,
        # Hyperbolic Functions
        'sinh': 1,
        'cosh': 1,
        'tanh': 1,
        'coth': 1,
        'sech': 1,
        'csch': 1,
        # Hyperbolic Inverses
        'asinh': 1,
        'acosh': 1,
        'atanh': 1,
        'acoth': 1,
        'asech': 1,
        'acsch': 1,
        # Derivative
        'derivative': 2,
        # custom functions
        'f': 1,
        'g': 2,
        'h': 3,
    }

symbols = ['I', 'INT+', 'INT-', 'INT', 'FLOAT', '-', '.', '10^']

In [8]:
constants = ['pi', 'E']
variables = ['x', 'y', 'z', 't', 'Y', "Y'", "Y''"]
functions = ['f', 'g', 'h']
elements = [str(i) for i in range(-10, 10)]
coefficients = [f'a{i}' for i in range(10)]

In [9]:
no_child_symbols = constants + variables + functions + elements + coefficients

### Черновик для создания adj matrices

In [18]:
from tqdm import tqdm
import queue

def get_ancestors(exp_list, exp_len):
    q = queue.LifoQueue()
    q.put(-1)                            # so last element gets this parent but doesn't save it

    ancestors = {0: []}
    node2parent = {}
    levels = {0: -1}

    parent = 0
    for i in range(exp_len):
        op_now = exp_list[i]

        node2parent[i] = parent
        try:
            levels[i] = levels[parent] + 1
        except:
            print('you are in except')
            return {}, {}


        if op_now in OPERATORS or op_now in symbols:   # <=> node has children
            if op_now in OPERATORS and OPERATORS[op_now] == 2:    # <=> node has 2 children
                q.put(i)
            parent = i
        elif op_now in no_child_symbols:
            if op_now.isdigit() and i + 1 < exp_len and exp_list[i + 1].isdigit():   # e.g. 18
                parent = i
            else:
                parent = q.get()
        else:
            print(op_now)
            #raise(NotFound)
            return False
        ancestors[i] = [i] + ancestors[node2parent[i]]

    return ancestors, levels

In [19]:
def get_path(i, j):
    if i == j:
        return "<self>"
    anc_i = set(ancestors[i])

    for node in ancestors[j][-(levels[i] + 1) :]:
        if node in anc_i:
            up_n = levels[i] - levels[node]
            down_n = levels[j] - levels[node]
            return str(round(up_n + 0.001 * down_n, 5))

In [20]:
def get_ud_masks(ancestors, levels, exp_len):
    path_rels = []
    for i in range(exp_len):
        path_rels.append(" ".join([get_path(i, j) for j in range(exp_len)]))
    
    return path_rels

###

In [21]:
F_prefix  = ['add', 'mul', 'INT-', '3', 'Y', 'mul', 'x', "Y'"]  # mul a8 pow x INT+ 3

In [22]:
ancestors, levels = get_ancestors(F_prefix, len(F_prefix))
rel_matrix = get_ud_masks(ancestors, levels, len(F_prefix))
rel_matrix

['<self> 0.001 0.002 0.003 0.002 0.001 0.002 0.002',
 '1.0 <self> 0.001 0.002 0.001 1.001 1.002 1.002',
 '2.0 1.0 <self> 0.001 1.001 2.001 2.002 2.002',
 '3.0 2.0 1.0 <self> 2.001 3.001 3.002 3.002',
 '2.0 1.0 1.001 1.002 <self> 2.001 2.002 2.002',
 '1.0 1.001 1.002 1.003 1.002 <self> 0.001 0.001',
 '2.0 2.001 2.002 2.003 2.002 1.0 <self> 1.001',
 '2.0 2.001 2.002 2.003 2.002 1.0 1.001 <self>']

### подсчет сколько брать tree_rel_vocab_size

In [28]:
#import jsonlines
import json
from tqdm import tqdm

In [29]:
from collections import Counter

def _insert(iterable):
    words = []
    for w in iterable:
        words.append(w)
    word_count.update(words)

word_count = Counter()
with open("data/", 'r') as f:
    for line in tqdm(f):
        matrix = json.loads(line)
        new_matrix = [line.split() for line in matrix]
        for tokens in new_matrix:
            for elem in tokens:
                _insert(elem.split("_"))

45339871it [11:49:50, 1064.56it/s]


In [30]:
len(word_count)

26317

In [37]:
sorted(list(word_count.values()))[::-1][:1000][-10:]

[1120, 1120, 1112, 1112, 1111, 1111, 1106, 1106, 1104, 1104]

In [38]:
sorted(list(word_count.values()))[::-1][:2000][-10:]

[219, 219, 218, 218, 218, 218, 217, 217, 217, 217]

In [39]:
sorted(list(word_count.values()))[::-1][:5000][-10:]

[29, 29, 29, 29, 29, 28, 28, 28, 28, 28]

In [40]:
sorted(list(word_count.values()))[::-1][:10000][-10:]

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

In [45]:
sorted(list(word_count.values()))[::-1][:11000][-10:]

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6]

In [46]:
sorted(list(word_count.values()))[::-1][:12000][-10:]

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]

### Generating tree_rel_vocab

In [34]:
special_tokens = "unk"
num_spec_tokens = len(special_tokens.split("_"))
# -2 to reserve spots for PAD and UNK token
dict_size = 500
dict_size = dict_size - num_spec_tokens if dict_size and dict_size > num_spec_tokens else dict_size
most_common = word_count.most_common(dict_size)

NameError: name 'word_count' is not defined

In [40]:
values = np.array(list(word_count.values()))
keys = np.array(list(word_count.keys()))
idxs = np.argsort(values)[::-1]
for i, (key, value) in enumerate(zip(keys[idxs], values[idxs])):
    print(value, key, i)

215235 <self> 0
205250 0.001 1
205250 1.0 2
195265 0.002 3
195265 2.0 4
185280 0.003 5
185280 3.0 6
180218 2.002 7
165652 4.0 8
165652 0.004 9
157936 2.003 10
157936 3.002 11
149647 2.001 12
149647 1.002 13
135328 0.005 14
135328 5.0 15
126700 1.003 16
126700 3.001 17
126184 3.003 18
125676 1.001 19
110286 2.004 20
110286 4.002 21
98612 0.006 22
98612 6.0 23
93767 4.001 24
93767 1.004 25
75788 4.003 26
75788 3.004 27
68211 2.005 28
68211 5.002 29
63435 0.007 30
63435 7.0 31
61036 1.005 32
61036 5.001 33
42788 3.005 34
42788 5.003 35
37129 6.002 36
37129 2.006 37
35786 8.0 38
35786 0.008 39
35132 4.004 40
34561 1.006 41
34561 6.001 42
21927 3.006 43
21927 6.003 44
18021 2.007 45
18021 7.002 46
17859 0.009000000000000001 47
17859 9.0 48
17232 1.007 49
17232 7.001 50
16357 5.004 51
16357 4.005 52
10241 3.007 53
10241 7.003 54
8004 0.01 55
8004 10.0 56
7820 2.008 57
7820 8.002 58
7687 1.008 59
7687 8.001 60
7364 6.004 61
7364 4.006 62
6746 5.005 63
4343 3.008 64
4343 8.003 65
3315 0.011 66

In [42]:
values = np.array(list(word_count.values()))
keys = np.array(list(word_count.keys()))
idxs = np.argsort(values)[::-1]
vocab = []
for i, (key, value) in enumerate(zip(keys[idxs], values[idxs])):
    print(value, key, i)
    vocab.append(key)
with open("rel_vocab.txt", "w") as fout:
    fout.write("\n".join(vocab))

215235 <self> 0
205250 0.001 1
205250 1.0 2
195265 0.002 3
195265 2.0 4
185280 0.003 5
185280 3.0 6
180218 2.002 7
165652 4.0 8
165652 0.004 9
157936 2.003 10
157936 3.002 11
149647 2.001 12
149647 1.002 13
135328 0.005 14
135328 5.0 15
126700 1.003 16
126700 3.001 17
126184 3.003 18
125676 1.001 19
110286 2.004 20
110286 4.002 21
98612 0.006 22
98612 6.0 23
93767 4.001 24
93767 1.004 25
75788 4.003 26
75788 3.004 27
68211 2.005 28
68211 5.002 29
63435 0.007 30
63435 7.0 31
61036 1.005 32
61036 5.001 33
42788 3.005 34
42788 5.003 35
37129 6.002 36
37129 2.006 37
35786 8.0 38
35786 0.008 39
35132 4.004 40
34561 1.006 41
34561 6.001 42
21927 3.006 43
21927 6.003 44
18021 2.007 45
18021 7.002 46
17859 0.009000000000000001 47
17859 9.0 48
17232 1.007 49
17232 7.001 50
16357 5.004 51
16357 4.005 52
10241 3.007 53
10241 7.003 54
8004 0.01 55
8004 10.0 56
7820 2.008 57
7820 8.002 58
7687 1.008 59
7687 8.001 60
7364 6.004 61
7364 4.006 62
6746 5.005 63
4343 3.008 64
4343 8.003 65
3315 0.011 66

## Generating adj matrices

In [35]:
import json
#import jsonlines

for set_name in ['valid', 'test', 'no_dups_valtest']:
    with open('data/ode1.' + set_name, 'r') as expressions:
        with open('data/ODE_rel_matrix_'+set_name+'.json', 'w') as rel_matrix_json:
            for i, line in tqdm(enumerate(expressions)):
                #print(line)
                qa = line.split('|')[1].split('\t')
                if len(qa) == 2:
                    q, a = qa
                else:
                    print(i,'is broken')
                    continue
                #print(q, ';', a)
                
                q = q.split()
                #a = a.split()
                
                ance_lev = get_ancestors(q, len(q))
                if len(ance_lev) == 2:
                    ancestors, levels = ance_lev
                else:
                    print(i, 'is broken')
                    continue
                rel_matrix_q = get_ud_masks(ancestors, levels, len(q))              

                #ancestors, levels = get_ancestors(a, len(a))
                #rel_matrix_a = get_ud_masks(ancestors, levels, len(a))
                
                rel_matrix_json.write(json.dumps(rel_matrix_q) + '\n')

7994it [12:11, 10.93it/s]
7968it [11:06, 11.95it/s]
13797it [20:57, 10.97it/s]


In [36]:
print(1)

1


### Черновик для создания root paths

In [34]:
F_prefix = ['add', 'mul', 'INT-', '3', 'Y', 'mul', 'x', "Y'"]  # mul a8 pow x INT+ 3

In [35]:
from tqdm import tqdm
import queue
from collections import OrderedDict, deque

def get_root_paths(exp_list, exp_len):
    q = deque()
    q.append(-1)                            # so last element gets this parent but doesn't save it

    root_paths = OrderedDict([(i, '') for i in range(-1, len(exp_list) + 1)])   # init with empty lines to be able to add
    prev_is_digit = False
    is_right, is_down = False, False

    parent = 0
    for i in range(exp_len):
        op_now = exp_list[i]
        
        if prev_is_digit:
            if op_now.isdigit():   # e.g. 18
                parent = i - 1
            else:
                parent = q.pop()
                is_right = True
            prev_is_digit = False
            
        if i != 0:
            if parent == -1:
                print('ohoh')
                return -1
            root_paths[i] += root_paths[parent]
            if is_right:
                last_step = '2'     # right
            elif is_down:
                last_step = '0'     # down
            else:
                last_step = '1'     # left

            root_paths[i] += last_step
            is_right, is_down = False, False
        

        if op_now in OPERATORS or op_now in symbols:   # <=> node has children
            if op_now in OPERATORS and OPERATORS[op_now] == 2:    # <=> node has 2 children
                q.append(i)
            else:
                is_down = True
            parent = i
        elif op_now in no_child_symbols:
            if op_now.isdigit() and i + 1 < exp_len:
                prev_is_digit = True
            else:
                parent = q.pop()
                is_right = True

                
    return root_paths

In [36]:
root_paths = get_root_paths(F_prefix, len(F_prefix))

root_paths

OrderedDict([(-1, ''),
             (0, ''),
             (1, '1'),
             (2, '11'),
             (3, '110'),
             (4, '12'),
             (5, '2'),
             (6, '21'),
             (7, '22'),
             (8, '')])

In [37]:
root_paths = get_root_paths(F_prefix, len(F_prefix))



for i, key in enumerate(root_paths):
    if root_paths[key] == '':
        print('root')
    else:
        print('/'.join(root_paths[key]))

root
root
1
1/1
1/1/0
1/2
2
2/1
2/2
root


### Реал пишу в файлы:

In [ ]:
for set_name in ['test', 'valid', 'train']:
    with open('data/ode1.' + set_name, 'r') as expressions:
        with open('data/ODE_root_paths_'+set_name+'.json', 'w') as root_paths:
            first = True
            for i, line in tqdm(enumerate(expressions)):
                # print(line)
                qa = line.split('|')[1].split('\t')
                if qa.__len__() != 2:
                    print('line is broken', i)
                    continue
                q, a = qa
                q = q.split()
                #a = a.split()
                
                paths_q = get_root_paths(q, len(q))
                # paths_a = get_root_paths(a, len(a))
                if paths_q == -1:  # or paths_a == -1:
                    print('line is broken', i)
                    continue
                paths_q = [paths_q[path] if paths_q[path] != '' else 'root' for i, path in enumerate(paths_q)]
                #paths_a = [paths_a[path] if paths_a[path] != '' else 'root' for i, path in enumerate(paths_a)]
                root_paths.write(("\n" if not first else "") + " ".join(paths_q)) # + "\t" + " ".join(paths_a))
                first = False
                # if i >= 5: break
            root_paths.write("\n")

7968it [00:01, 6893.25it/s]
7994it [00:01, 6955.75it/s]
1324764it [01:00, 19456.37it/s]

## Checking same to const in train/val/test

In [5]:
pip install psutil --user

Note: you may need to restart the kernel to use updated packages.


In [6]:
import psutil
import sys
from tqdm import tqdm

print(psutil.virtual_memory().percent)

21.3


In [11]:
constants = set(elements)
print(constants)

{'-10', '9', '5', '8', '-4', '-5', '6', '-8', '1', '-7', '3', '0', '7', '-9', '2', '-6', '-1', '-2', '-3', '4'}


### QA

In [12]:
def remove_const_in_line(line, cleaned_str):
    line = line.split()
    for j in range(len(line) - 1):
        word = line[j]
        if word in constants:
            if line[j+1] in constants:
                continue
            else:
                cleaned_str += ' C'
        else:
            cleaned_str += ' '
            cleaned_str += word
    if line[-1] in constants:
        cleaned_str += ' C'
    
    return cleaned_str

In [13]:
def how_many_in_set(path, sets, ALL_TRAIN_SET, same_lines_file):
    
    same_lines_num = 0

    for set_name in sets:
        with open(path + set_name, 'r') as expressions:
            for i, line in tqdm(enumerate(expressions)):
                qa = line.split('|')[1].split('\t')
                if len(qa) == 2:
                    q, a = qa
                else:
                    print(i,'is broken')
                    continue

                cleaned_str = ''
                cleaned_str = remove_const_in_line(q, cleaned_str)

                cleaned_str += '\t'
                cleaned_str += remove_const_in_line(a, cleaned_str)

                if cleaned_str in ALL_TRAIN_SET:
                    #print(i, q, a)
                    #print(cleaned_str)
                    #print()
                    
                    if same_lines_file is not None:
                        print(set_name, i, file=same_lines_file)
                    same_lines_num += 1


    print(same_lines_num)

In [14]:
ALL_TRAIN_SET = set()

with open('data/ode1.' + 'train', 'r') as expressions:
    for i, line in tqdm(enumerate(expressions)):
        qa = line.split('|')[1].split('\t')
        if len(qa) == 2:
            q, a = qa
        else:
            print(i,'is broken')
            continue

        cleaned_str = ''
        cleaned_str = remove_const_in_line(q, cleaned_str)

        cleaned_str += '\t'
        cleaned_str += remove_const_in_line(a, cleaned_str)

        ALL_TRAIN_SET.add(cleaned_str)
        
print(sys.getsizeof(ALL_TRAIN_SET))

65340431it [1:22:16, 13236.74it/s]

2147483880


In [15]:
len(ALL_TRAIN_SET)

58137738

In [16]:
with open('data/ODE_same_lines_qa.txt', 'w') as same_lines_file:
    how_many_in_set(path='data/ode1.', 
                    sets=['valid', 'test'], 
                    ALL_TRAIN_SET=ALL_TRAIN_SET,
                    same_lines_file=same_lines_file)

7994it [00:00, 12191.63it/s]
7968it [00:00, 12626.07it/s]

2165


#### Добавляем в ALL_TRAIN_SET значения из val и test для генерации своих совсем новых:

In [17]:
for set_now in ['valid', 'test']:
    with open('data/ode1.' + set_now, 'r') as expressions:
        for i, line in tqdm(enumerate(expressions)):
            qa = line.split('|')[1].split('\t')
            if len(qa) == 2:
                q, a = qa
            else:
                print(i,'is broken')
                continue

            cleaned_str = ''
            cleaned_str = remove_const_in_line(q, cleaned_str)

            cleaned_str += '\t'
            cleaned_str += remove_const_in_line(a, cleaned_str)

            ALL_TRAIN_SET.add(cleaned_str)

    print(sys.getsizeof(ALL_TRAIN_SET))

7994it [00:00, 13111.44it/s]
1071it [00:00, 10709.98it/s]

2147483880


7968it [00:00, 11395.58it/s]

2147483880


In [18]:
len(ALL_TRAIN_SET)

58151535

In [19]:
58151535 - 58137738

13797

добавилось сколько и ждали

### Создаем no_dups_valtest: 

In [20]:
set2same_lines_num = {'valid':[], 'test':[]}

with open('data/' + 'ODE_same_lines_qa.txt', 'r') as same_lines_file:
    for line in tqdm(same_lines_file):
        set_name, str_num = line.split()
        set2same_lines_num[set_name].append(int(str_num))

2165it [00:00, 100229.23it/s]


In [21]:
with open('data/ode1.' + 'no_dups_valtest', 'w') as no_dups_file:
    
    for set_name in ['valid', 'test']:
        with open('data/ode1.' + set_name, 'r') as expressions:
                for i, line in tqdm(enumerate(expressions)):
                    if i not in set2same_lines_num[set_name]:
                        print(line, file=no_dups_file, end='')

7994it [00:00, 39415.08it/s]
7968it [00:00, 42184.19it/s]


In [23]:
!wc -l 'data/ode1.no_dups_valtest'

13797 data/ode1.no_dups_valtest


In [3]:
!wc -l 'data/ode1.valid'

7994 data/ode1.valid


In [4]:
!wc -l 'data/ode1.test'

7968 data/ode1.test


In [5]:
7994+7968

15962

In [6]:
13797/15962

0.8643653677484024